In [17]:
from statsmodels.tsa.stattools import adfuller
import yfinance as yf

close_series = amzn['Adj Close'] if 'Adj Close' in amzn.columns else amzn['Close']
result = adfuller(close_series.dropna(), autolag='AIC')
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

ADF Statistic: 0.6734836882159839
p-value: 0.9892935605889442
Critical Values: {'1%': -3.4320982741938835, '5%': -2.8623124910596176, '10%': -2.567181207712407}


In [320]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# import pandas_ta
# import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()
end_date = '2025-06-01'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date, auto_adjust = False).stack()
df.columns =  df.columns.str.lower()
df

[*********************100%***********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2017-06-05 A        56.852474   60.400002   60.840000   60.330002   60.799999   
           AAPL     35.854855   38.482498   38.612499   38.365002   38.584999   
           ABBV     47.947418   67.510002   67.860001   66.879997   67.250000   
           ABT      40.446308   46.619999   46.750000   46.380001   46.570000   
           ACGL     31.167286   32.776669   33.043331   32.700001   32.863335   
...                       ...         ...         ...         ...         ...   
2025-05-30 XYL     126.040001  126.040001  126.690002  125.199997  126.519997   
           YUM     143.940002  143.940002  144.479996  143.089996  144.039993   
           ZBH      92.169998   92.169998   93.040001   91.440002   92.940002   
           ZBRA    289.769989  289.769989  290.559998  283.920013  289.929993   
           ZTS     168.630005  168.630005  169.789993  165.339996  166.919998   

Price                   volume  
Date       Ticker               
2017-06-05 A         1473300.0  
           AAPL    101326800.0  
           ABBV      4918100.0  
           ABT       7810600.0  
           ACGL      2004900.0  
...                        ...  
2025-05-30 XYL       2624100.0  
           YUM       3184400.0  
           ZBH       3505200.0  
           ZBRA       656100.0  
           ZTS       5467800.0  

[990176 rows x 6 columns]

#Technical indicators for each stock
Garman-klass volatility
(calculates volatility of a stock based on OHLC (open high low close) usually daily
Benefit - incoporates intraday variability informaiton to estimate volatility
con - volatility is not robust for opening jumps in prices

RSI(momentum indicator)
most useful in an oscillating market(between bullish and bearish)
lookback period of usually 14 days
generally >70 means overbought(bearish)
          <30 means oversold(bullish)
con - false positives(bullish crossover follwoed by a sudden decline in stock
    - false negative(bearish crossover followed by sudden increase in stock)

bollinger bands
middle band -20day SMA of closing prices
upper/lower band - adding/subtracting 2 standard dev from middle band
generally touching/going above upper band indicates overbought
touching/going lower lower band indicates oversold
widening band means increase in volatility (stdv of price increases)
tigthtening band(squeeze) means decrease volatility but indicates a potential breakout
use with momentum indicators

ATR(averager true range)
ATR = (prev ATR(n-1)+TR)/n
TR = (1/n) * summation((max(High-low, |high-yst close|, |low - ystclose|)) overt n days
con - no subjective measure, only measures volatility and not direction of asset price
if ATR is $1, price has an avg range of movement $1 per trading day

MACD
MACD=(12-Period EMA) − (26-Period EMA)
a negative macd indicates a bearish signal 
a positive macd indicates a bullish signal
cons - False positives 

Dollar Volume
closing price * total volume traded
helps to determine if a stock is highly active and has high liquidity
helps to normalise stocks (eg filter out those w high trade volume but low price)

In [487]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open'])))*2
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x,length = 20))
#(level=1 means the first index (ticker)) transform applies the the rsi formula to all those in groupby(and keeps the shape the same) length is lookback period
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close= np.log1p(x),length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close= np.log1p(x),length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close= np.log1p(x),length=20).iloc[:,2])
#iloc picks the lower/middle/upper portion of data (log1p = log(1+x) to prevent log(0) stabilise variance
def compute_Zatr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std()) #z-score standardisation
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['close'],
                        length = 14)
    return atr
df['Zatr']= df.groupby(level=1, group_keys=False).apply(compute_Zatr)
df['atr'] = df.groupby(level=1, group_keys= False).apply(compute_atr)
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
df['macd'] = df.groupby(level=1,group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
Date       Ticker                                                               
2017-06-05 A        56.852474   60.400002   60.840000   60.330002   60.799999   
           AAPL     35.854855   38.482498   38.612499   38.365002   38.584999   
           ABBV     47.947418   67.510002   67.860001   66.879997   67.250000   
           ABT      40.446308   46.619999   46.750000   46.380001   46.570000   
           ACGL     31.167286   32.776669   33.043331   32.700001   32.863335   
...                       ...         ...         ...         ...         ...   
2025-05-30 XYL     126.040001  126.040001  126.690002  125.199997  126.519997   
           YUM     143.940002  143.940002  144.479996  143.089996  144.039993   
           ZBH      92.169998   92.169998   93.040001   91.440002   92.940002   
           ZBRA    289.769989  289.769989  290.559998  283.920013  289.929993   
           ZTS     168.630005  168.630005  169.789993  165.339996  166.919998   

Price                   volume  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                       
2017-06-05 A         1473300.0          0.051899        NaN       NaN   
           AAPL    101326800.0          0.056717        NaN       NaN   
           ABBV      4918100.0          0.261482        NaN       NaN   
           ABT       7810600.0          0.108952        NaN       NaN   
           ACGL      2004900.0          0.040993        NaN       NaN   
...                        ...               ...        ...       ...   
2025-05-30 XYL       2624100.0          0.003007  57.889330  4.800087   
           YUM       3184400.0          0.000583  44.883266  4.967293   
           ZBH       3505200.0          0.006578  40.059086  4.505226   
           ZBRA       656100.0          0.000694  55.497931  5.520376   
           ZTS       5467800.0         -0.007522  59.546814  5.034332   

Price                bb_mid   bb_high      Zatr       atr      macd  \
Date       Ticker                                                     
2017-06-05 A            NaN       NaN       NaN       NaN       NaN   
           AAPL         NaN       NaN       NaN       NaN       NaN   
           ABBV         NaN       NaN       NaN       NaN       NaN   
           ABT          NaN       NaN       NaN       NaN       NaN   
           ACGL         NaN       NaN       NaN       NaN       NaN   
...                     ...       ...       ...       ...       ...   
2025-05-30 XYL     4.834142  4.868197  0.189565  2.279783  1.029981   
           YUM     4.990759  5.014225  0.681439  2.557451 -0.801297   
           ZBH     4.561076  4.616926 -0.238112  2.597415 -0.926024   
           ZBRA    5.648873  5.777371  0.168361  9.225290  0.967280   
           ZTS     5.088457  5.142582  0.552765  4.026517  1.022106   

Price              dollar_volume  
Date       Ticker                 
2017-06-05 A           83.760750  
           AAPL      3633.057679  
           ABBV       235.810198  
           ABT        315.909934  
           ACGL        62.487292  
...                          ...  
2025-05-30 XYL        330.741566  
           YUM        458.362544  
           ZBH        323.074278  
           ZBRA       190.118090  
           ZTS        922.035141  

[990176 rows x 15 columns]

In [ ]:
SORTING DF BY MONTH AND GETTING MEAN OF INDICATORS AND DOLLAR VOL FOR THAT MONTH

Last columns to get values of indicators
resample('M') sorts by month shows by end date('31')/ 'MS' sorts by month but shows the start date (01)
last() returns last day of month
.dropna removes entire month that contains any NaN 

In [489]:
last_columns = [c for c in df.columns.unique(0) if c not in ['dollar_volume','volume','open','close','high','low']]
data = (pd.concat([df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume'),
            df.unstack()[last_columns].resample('M').last().stack('Ticker')],axis =1)).dropna()
data

dollar_volume   adj close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2017-07-31 A           92.018441   56.403656          0.051966  46.738762   
           AAPL      2915.393360   34.643608          0.060804  49.136372   
           ABBV       258.186406   50.098270          0.264213  43.449585   
           ABT        247.002300   42.903431          0.112306  54.130561   
           ACGL        42.369787   30.828129          0.028633  53.516378   
...                          ...         ...               ...        ...   
2025-05-31 XYL        165.032884  126.040001          0.003007  57.889330   
           YUM        274.965077  143.940002          0.000583  44.883266   
           ZBH        283.996451   92.169998          0.006578  40.059086   
           ZBRA       163.412584  289.769989          0.000694  55.497931   
           ZTS        515.972684  168.630005         -0.007522  59.546814   

                     bb_low    bb_mid   bb_high      Zatr       atr      macd  
Date       Ticker                                                              
2017-07-31 A       4.034275  4.060711  4.087147 -1.663907  0.801870 -0.049060  
           AAPL    3.528764  3.570535  3.612307 -1.330006  0.584934 -0.156441  
           ABBV    3.933486  3.964924  3.996362 -1.425244  1.190566 -0.142871  
           ABT     3.739666  3.782008  3.824349 -1.670182  0.643138  0.143963  
           ACGL    3.423910  3.453108  3.482306 -1.121348  0.474107 -0.009988  
...                     ...       ...       ...       ...       ...       ...  
2025-05-31 XYL     4.800087  4.834142  4.868197  0.189565  2.279783  1.029981  
           YUM     4.967293  4.990759  5.014225  0.681439  2.557451 -0.801297  
           ZBH     4.505226  4.561076  4.616926 -0.238112  2.597415 -0.926024  
           ZBRA    5.520376  5.648873  5.777371  0.168361  9.225290  0.967280  
           ZTS     5.034332  5.088457  5.142582  0.552765  4.026517  1.022106  

[46818 rows x 10 columns]

SORTING BY DOLLAR_VOL AMOUNT BY GETTING THE VOL_RANK THEN FILTERING TO GET THE TOP 150

rolling creats a window (eg 5 yrs 5*12) that u can call a func on (like.mean(),.sum,.std), takes in min_period= min period for data, default is window
rank like sorted


In [491]:
data['dollar_volume']=(data.loc[:,'dollar_volume'].unstack('Ticker').rolling(5*12,min_periods = 12).mean().stack())
data['dollar_vol_rank'] = data.groupby('Date')['dollar_volume'].rank(ascending=False)
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume','dollar_vol_rank'],axis = 1)
data

adj close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2018-06-30 AAPL     43.787849          0.047901  49.582363  3.782213   
           ABBV     68.034241          0.240378  38.080480  4.215851   
           ABT      53.980732          0.100747  47.018527  3.997916   
           ACN     147.388336          0.083522  58.053483  4.953232   
           ADBE    243.809998         -0.006006  50.937230  5.474781   
...                       ...               ...        ...       ...   
2025-05-31 VZ       43.959999         -0.010284  53.634958  3.773740   
           WDAY    247.710007         -0.019732  48.534163  5.453738   
           WFC      74.779999         -0.005912  57.092537  4.286886   
           WMT      98.720001         -0.013614  58.713276  4.569311   
           XOM     102.300003         -0.000989  42.393078  4.620994   

                     bb_mid   bb_high      Zatr       atr      macd  
Date       Ticker                                                    
2018-06-30 AAPL    3.819717  3.857222 -1.247905  0.729048 -0.279226  
           ABBV    4.278428  4.341006 -0.616421  2.046478 -1.006095  
           ABT     4.027670  4.057425 -1.304278  0.927787 -0.352055  
           ACN     4.985015  5.016799 -0.938710  2.912173  0.031895  
           ADBE    5.521728  5.568676 -1.119771  5.593132 -0.111611  
...                     ...       ...       ...       ...       ...  
2025-05-31 VZ      3.796168  3.818596 -0.328290  0.759696  0.066308  
           WDAY    5.556007  5.658276  0.711268  7.629883  0.017504  
           WFC     4.320663  4.354439  1.194300  1.690886  1.010235  
           WMT     4.588856  4.608402  2.703517  2.050799  0.952412  
           XOM     4.662747  4.704499  0.412755  2.162145 -0.987838  

[12516 rows x 9 columns]

CALCULATING MONTHLY RETURNS FOR DIFF TIME HORIZONS AS FEATURES
TIME HORIZONS  (1MNTH, 2MNTH,3MNTH,6MNTH,9MTNTH,12MNTH)
outlier_cutoff is to filter out those that are on the extreme ends
clip(lower,upper) sets all values below the 0.5th percentile to that percentile(0.5), upper is above (0.95 percentile)
pct_change(periods) = (curr - (val from periods ago))/(val from periods ago)
add(1) converts the percnt change to a growth factor (eg .5% to 1.05)
.pow(1/time) finds annualised growth factor (eg 1.15 for 3 mnths, annualised = (1.15)^(1/3)
sub(1) converts annualised growth factor to annualised return percentage

In [493]:
def calc_returns(df):
    outlier_cutoff = 0.005
    time_horizon = [1,2,3,6,9,12]
    for time in time_horizon:
        df[f'return_{time}months'] = (df['adj close']
                                      .pct_change(time)
                                      .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                             upper=x.quantile(1-outlier_cutoff)))
                                      .add(1)
                                      .pow(1/time)
                                      .sub(1))
    return df
data = data.groupby(level=1, group_keys=False).apply(calc_returns).dropna()
data

adj close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2019-06-30 AAPL     47.530712          0.034090  56.282051  3.781842   
           ABBV     55.922264          0.180414  42.046273  3.983328   
           ABT      75.733643          0.078471  64.665953  4.254858   
           ACN     169.383636          0.065014  58.459449  5.099023   
           ADBE    294.649994         -0.000336  58.679794  5.566200   
...                       ...               ...        ...       ...   
2025-05-31 VZ       43.959999         -0.010284  53.634958  3.773740   
           WDAY    247.710007         -0.019732  48.534163  5.453738   
           WFC      74.779999         -0.005912  57.092537  4.286886   
           WMT      98.720001         -0.013614  58.713276  4.569311   
           XOM     102.300003         -0.000989  42.393078  4.620994   

                     bb_mid   bb_high      Zatr       atr      macd  \
Date       Ticker                                                     
2019-06-30 AAPL    3.856704  3.931566 -1.109137  0.972634  0.024352   
           ABBV    4.084980  4.186631 -0.229499  2.455927 -0.933298   
           ABT     4.315521  4.376183 -0.951330  1.202359  0.937872   
           ACN     5.129683  5.160344 -1.069925  2.585643  0.162169   
           ADBE    5.645850  5.725500 -0.887026  6.703318  0.428373   
...                     ...       ...       ...       ...       ...   
2025-05-31 VZ      3.796168  3.818596 -0.328290  0.759696  0.066308   
           WDAY    5.556007  5.658276  0.711268  7.629883  0.017504   
           WFC     4.320663  4.354439  1.194300  1.690886  1.010235   
           WMT     4.588856  4.608402  2.703517  2.050799  0.952412   
           XOM     4.662747  4.704499  0.412755  2.162145 -0.987838   

                   return_1months  return_2months  return_3months  \
Date       Ticker                                                   
2019-06-30 AAPL          0.130519       -0.004965        0.015095   
           ABBV         -0.052014       -0.042929       -0.029421   
           ABT           0.104689        0.028136        0.018439   
           ACN           0.037626        0.005732        0.019101   
           ADBE          0.087671        0.009291        0.034051   
...                           ...             ...             ...   
2025-05-31 VZ           -0.002270       -0.007679        0.011968   
           WDAY          0.011061        0.029913       -0.020189   
           WFC           0.058869        0.023408       -0.013502   
           WMT           0.017570        0.061706        0.002093   
           XOM          -0.022605       -0.068285       -0.024827   

                   return_6months  return_9months  return_12months  
Date       Ticker                                                   
2019-06-30 AAPL          0.039950       -0.013236         0.006858  
           ABBV         -0.032749       -0.024888        -0.016205  
           ABT           0.026936        0.016743         0.028618  
           ACN           0.047516        0.011115         0.011659  
           ADBE          0.045016        0.009775         0.015908  
...                           ...             ...              ...  
2025-05-31 VZ            0.004157        0.011156         0.011000  
           WDAY         -0.001526       -0.006713         0.013277  
           WFC          -0.001336        0.029559         0.020651  
           WMT           0.012148        0.028495         0.035317  
           XOM          -0.020514       -0.012807        -0.008481  

[10323 rows x 15 columns]

In [475]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                             'famafrench',
                             start = '2010')[0].drop('RF',axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'Date'
factor_data = factor_data.join(data['return_1months']).sort_index()
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1months
Date       Ticker                                                        
2019-06-30 AAPL    0.0692  0.0037 -0.0067  0.0091 -0.0040        0.130519
           ABBV    0.0692  0.0037 -0.0067  0.0091 -0.0040       -0.052014
           ABT     0.0692  0.0037 -0.0067  0.0091 -0.0040        0.104689
           ACN     0.0692  0.0037 -0.0067  0.0091 -0.0040        0.037626
           ADBE    0.0692  0.0037 -0.0067  0.0091 -0.0040        0.087671
...                   ...     ...     ...     ...     ...             ...
2025-04-30 VZ     -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.013058
           WDAY   -0.0085 -0.0185 -0.0341 -0.0284 -0.0267        0.049116
           WFC    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.010865
           WMT    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267        0.107757
           XOM    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.111830

[10183 rows x 6 columns]

In [ ]:
Filter out stocks with less than 10 months of data

In [477]:
observations = factor_data.groupby(level=1).size()
valid_stocks = observations[observations >=10]
factor_data = factor_data[factor_data.index.get_level_values('Ticker').isin(valid_stocks.index)]
factor_data = factor_data.dropna()

In [479]:
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1months
Date       Ticker                                                        
2019-06-30 AAPL    0.0692  0.0037 -0.0067  0.0091 -0.0040        0.130519
           ABBV    0.0692  0.0037 -0.0067  0.0091 -0.0040       -0.052014
           ABT     0.0692  0.0037 -0.0067  0.0091 -0.0040        0.104689
           ACN     0.0692  0.0037 -0.0067  0.0091 -0.0040        0.037626
           ADBE    0.0692  0.0037 -0.0067  0.0091 -0.0040        0.087671
...                   ...     ...     ...     ...     ...             ...
2025-04-30 VZ     -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.013058
           WDAY   -0.0085 -0.0185 -0.0341 -0.0284 -0.0267        0.049116
           WFC    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.010865
           WMT    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267        0.107757
           XOM    -0.0085 -0.0185 -0.0341 -0.0284 -0.0267       -0.111830

[10105 rows x 6 columns]

In [443]:
ROLLING BETA (ORDINARY LEAST SQUARES(OLS))
rollingOLS -> x : target variable ( 1mnth return of asset)
window = rolling window of up to 24 observations(2yrs) or number of yrs passed whichevers lower)
min_nob = minimun number of observations in window to compute a regression
betas calculated with ((X^T)*X)^-1 (X^T)y
y is [1mnth] X is the (x.drop) with intercept column
(x.columns for this = 5)

176

In [481]:
betas = (factor_data.groupby(level=1,
                             group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1months'],
                                     exog = sm.add_constant(x.drop('return_1months',axis=1)),
                                                            window = min(24,x.shape[0]),
                                                            min_nobs = len(x.columns)+1)
                                     .fit(params_only=True)
                                     .params
                                     .drop('const',axis=1)))
betas

Mkt-RF       SMB       HML       RMW       CMA
Date       Ticker                                                  
2019-06-30 AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
           ADBE         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2025-04-30 VZ      0.904803 -1.064601  0.905389  0.657994 -0.070619
           WDAY    1.108715 -0.937077  0.037348 -1.896113 -0.421493
           WFC     0.877759 -0.095225  0.559151 -1.655162 -0.085206
           WMT     0.787344  0.091225 -0.722478  0.394634  0.633478
           XOM     0.476822 -0.287361  1.039671  1.215638  0.114550

[10105 rows x 5 columns]

In [ ]:
** Take note betas are used to predict future retains (T+1) so shift down by 1 month
combining the beta values to data

In [483]:
betas = betas.groupby('Ticker').shift()

In [505]:
# five_factors = ['Mkt-RF','SMB','HML','RMW','CMA']
# data = data.join(betas)
# data.loc[:,five_factors] = data.groupby('Ticker',group_keys=False)[five_factors].apply(lambda x : x.fillna(x.mean()))
# data = data.dropna()
# data = data.drop('adj close', axis = 1)
data = data.drop('atr',axis = 1)

In [517]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10105 entries, (Timestamp('2019-06-30 00:00:00'), 'AAPL') to (Timestamp('2025-05-31 00:00:00'), 'XOM')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   garman_klass_vol  10105 non-null  float64
 1   rsi               10105 non-null  float64
 2   bb_low            10105 non-null  float64
 3   bb_mid            10105 non-null  float64
 4   bb_high           10105 non-null  float64
 5   atr               10105 non-null  float64
 6   macd              10105 non-null  float64
 7   return_1months    10105 non-null  float64
 8   return_2months    10105 non-null  float64
 9   return_3months    10105 non-null  float64
 10  return_6months    10105 non-null  float64
 11  return_9months    10105 non-null  float64
 12  return_12months   10105 non-null  float64
 13  Mkt-RF            10105 non-null  float64
 14  SMB               10105 non-null  float64
 15  HML       